# Scrap and Clean the Toronto DF

In [1]:
#install libraries
!conda install -c conda-forge lxml --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.1
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [2]:
# importing libraries
import pandas as pd
import lxml
print("libraries imported")

libraries imported


In [3]:
# Read Url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_list = pd.read_html(url)
len(df_list)

3

In [4]:
df_list[0]

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [5]:
toronto_hoods_df = df_list[0]
toronto_hoods_df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


## Cleaning the data

In [6]:
# drop the cells where Borough is not assigned
toronto_hoods_df_clean = toronto_hoods_df.loc[toronto_hoods_df['Borough'] != 'Not assigned'].reset_index(drop=True)

In [7]:
toronto_hoods_df_clean.head(5)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
# verify if any duplicate postal code exist if True we 
# aggregating  neighborshoods with same Postal code
#t_df =toronto_hoods_df_clean.groupby('Postal Code').agg(','.join)

if len (toronto_hoods_df_clean['Postal Code'].unique()) == toronto_hoods_df_clean.shape[0]:
    print("No need to agg the Postal Code")
else:
    toront_hoods_df_agg = toronto_hoods_df_clean.groupby('Postal Code').agg(','.join).reset_index()



No need to agg the Postal Code


In [9]:
#find if any neighborhood is Not assigned and assign name same as Borough 
if len(toronto_hoods_df_clean[toronto_hoods_df_clean['Neighbourhood']=='Not assigned']) > 0:
    cond_index = toronto_hoods_df_clean.loc[toronto_hoods_df_clean['Neighbourhood']=='Not assigned'].index
    ll=list(cond_index)
    for i in range(len(ll)):
        toronto_hoods_df_clean.at[ll[i],'Neighbourhood']=toronto_hoods_df_clean.at[ll[i],'Borough']
else:
    print("All neighborshoods are assigned")
                              

All neighborshoods are assigned


### show size of my Dataframe

In [10]:
toronto_hoods_df_clean.shape

(103, 3)

## adding Geo Coordinates to the dataframe

In [11]:
#install geocoder
#!pip3 install geocoder
!conda install -c conda-forge geocoder --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.1
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [12]:
# Get the coordinates of different area codes
import geocoder 

#initializa list containing Longitudes and latitude
longi_list =[]
lati_list=[]
for code in toronto_hoods_df_clean['Postal Code']:
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(code),key='API key')
        lat_lng_coords = g.latlng
        print("request for {}".format(code))
    #populate data in longitude and latitude lists
    lati_list.append(lat_lng_coords[0])
    longi_list.append(lat_lng_coords[1])
print("Geocoder complete")
    

request for M3A
request for M4A
request for M5A
request for M6A
request for M7A
request for M9A
request for M1B
request for M3B
request for M4B
request for M5B
request for M6B
request for M9B
request for M1C
request for M3C
request for M4C
request for M5C
request for M6C
request for M9C
request for M1E
request for M4E
request for M5E
request for M6E
request for M1G
request for M4G
request for M5G
request for M6G
request for M1H
request for M2H
request for M3H
request for M4H
request for M5H
request for M6H
request for M1J
request for M2J
request for M3J
request for M4J
request for M5J
request for M6J
request for M1K
request for M2K
request for M3K
request for M4K
request for M5K
request for M6K
request for M1L
request for M2L
request for M3L
request for M4L
request for M5L
request for M6L
request for M9L
request for M1M
request for M2M
request for M3M
request for M4M
request for M5M
request for M6M
request for M9M
request for M1N
request for M2N
request for M3N
request for M4N
request 

In [19]:
# Populate the DF with the longitude and latitude info
toronto_hoods_df_clean['Latitude']=lati_list
toronto_hoods_df_clean['Longitude']=longi_list
#toronto_hoods_df_clean.to_csv("Toronto_hood_with_coordinate.csv")
toronto_hoods_df_clean

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
